In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


client_number = 14
data_path = f"client_{client_number}.pt"  # Each client has its own file
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Partitioned dataset {data_path} not found!")

client_data = torch.load(data_path)
client_train_data, client_train_targets = client_data['data'], client_data['targets']
assert len(client_train_data) > 0, f"Client {client_number} has no data!"

batch_size = 128
client_train_loader = DataLoader(
    TensorDataset(client_train_data, client_train_targets),
    batch_size=batch_size,
    shuffle=True
)


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes*block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_features=False):
        x = self.conv1(x)
        x = self.bn1(x)
        out = F.relu(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        feature = out.view(out.size(0), -1)
        out = self.linear(feature)

        if return_features:
            return out, feature
        else:
            return out

def resnet18(num_classes=100):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

# For completeness, you have other definitions like resnet34, etc.
# but we focus on resnet18 as indicated.

class CNNCifar(nn.Module):
    def __init__(self):
        super(CNNCifar, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(128, 128, 3)
        self.conv3 = nn.Conv2d(128, 128, 3)
        self.fc1 = nn.Linear(128*4*4, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 128*4*4)
        x = self.fc1(x)
        return x


# local_model is your baseline model for alignment
local_model = resnet18(num_classes=100).to(device)

# Instead of a single neighbor_model, we will
# now load each neighbor's model inside a loop.


def get_layer_type(layer: nn.Module) -> str:
    return layer.__class__.__name__

def get_layer_dim(layer: nn.Module) -> tuple:
    if isinstance(layer, nn.Conv2d):
        return (layer.in_channels, layer.out_channels, layer.kernel_size)
    elif isinstance(layer, nn.Linear):
        return (layer.in_features, layer.out_features)
    elif isinstance(layer, nn.BatchNorm2d):
        return (layer.num_features,)
    else:
        if hasattr(layer, 'weight') and layer.weight is not None:
            return tuple(layer.weight.shape)
        return ()

def build_layer_info_dict(model: nn.Module) -> dict:
    layer_info = {}
    for name, layer in model.named_modules():
        if isinstance(layer, (nn.Conv2d, nn.Linear, nn.BatchNorm2d)):
            layer_type = get_layer_type(layer)
            layer_dim = get_layer_dim(layer)
            layer_info[name] = (layer_type, layer_dim, layer)
    return layer_info

def find_shared_and_unshared_layers(local_model, neighbor_model):
    local_dict = build_layer_info_dict(local_model)
    neighbor_dict = build_layer_info_dict(neighbor_model)

    shared_layers = []
    matched_neighbor_layers = set()

    for local_name, (ltype_local, ldim_local, _) in local_dict.items():
        for neighbor_name, (ltype_neighbor, ldim_neighbor, _) in neighbor_dict.items():
            if neighbor_name not in matched_neighbor_layers:
                if (ltype_local == ltype_neighbor) and (ldim_local == ldim_neighbor):
                    shared_layers.append((neighbor_name, local_name))
                    matched_neighbor_layers.add(neighbor_name)
                    break

    unshared_layers = [
        n_name for n_name in neighbor_dict
        if n_name not in matched_neighbor_layers
    ]
    return shared_layers, unshared_layers

def get_submodule(model, layer_name):
    submods = layer_name.split('.')
    cur_mod = model
    for sm in submods:
        cur_mod = getattr(cur_mod, sm)
    return cur_mod


class StructuredPruning(nn.Module):
    def __init__(self, model, shared_layers, unshared_layers,
                 lambda_reg=0.1, gamma_shared=0.05, gamma_unshared=0.02):
        super(StructuredPruning, self).__init__()
        self.model = model
        self.shared_layers = shared_layers
        self.unshared_layers = unshared_layers
        self.lambda_reg = lambda_reg
        self.gamma_shared = gamma_shared
        self.gamma_unshared = gamma_unshared

        self.shared_gating_params = nn.ParameterList()
        for (neighbor_name, _) in self.shared_layers:
            layer = get_submodule(self.model, neighbor_name)
            gating = nn.Parameter(torch.ones(layer.weight.shape[0], device=device))
            self.shared_gating_params.append(gating)

        self.unshared_gating_params = nn.ParameterList()
        for neighbor_name in self.unshared_layers:
            layer = get_submodule(self.model, neighbor_name)
            gating = nn.Parameter(torch.ones(layer.weight.shape[0], device=device))
            self.unshared_gating_params.append(gating)

    def forward(self, x):
        return self.model(x)

    def loss(self, local_model, inputs, targets):
        criterion = nn.CrossEntropyLoss()
        outputs = self.model(inputs)
        task_loss = criterion(outputs, targets)

        alignment_loss = 0.0
        for i, (neighbor_name, local_name) in enumerate(self.shared_layers):
            gating = self.shared_gating_params[i]
            neighbor_layer = get_submodule(self.model, neighbor_name)
            local_layer = get_submodule(local_model, local_name)

            if isinstance(neighbor_layer, nn.Conv2d):
                gating_reshaped = gating.view(-1,1,1,1)
                gated_weight = gating_reshaped * neighbor_layer.weight
                diff = gated_weight - local_layer.weight
                alignment_loss += self.lambda_reg * torch.norm(diff, p=2) / torch.numel(local_layer.weight)

            elif isinstance(neighbor_layer, nn.Linear):
                gating_reshaped = gating.view(-1,1)
                gated_weight = gating_reshaped * neighbor_layer.weight
                diff = gated_weight - local_layer.weight
                alignment_loss += self.lambda_reg * torch.norm(diff, p=2) / torch.numel(local_layer.weight)

            elif isinstance(neighbor_layer, nn.BatchNorm2d):
                gated_weight = gating * neighbor_layer.weight
                diff_weight = gated_weight - local_layer.weight
                if local_layer.bias is not None and neighbor_layer.bias is not None:
                    gated_bias = gating * neighbor_layer.bias
                    diff_bias = gated_bias - local_layer.bias
                    alignment_loss += self.lambda_reg * (
                        torch.norm(diff_weight, p=2) / torch.numel(local_layer.weight) +
                        torch.norm(diff_bias, p=2) / torch.numel(local_layer.bias)
                    )
                else:
                    alignment_loss += self.lambda_reg * (
                        torch.norm(diff_weight, p=2) / torch.numel(local_layer.weight)
                    )
            else:
                continue

        group_lasso_shared = 0.0
        for i, (neighbor_name, _) in enumerate(self.shared_layers):
            gating = self.shared_gating_params[i]
            layer = get_submodule(self.model, neighbor_name)

            if isinstance(layer, nn.Conv2d):
                gating_reshaped = gating.view(-1,1,1,1)
                gl = torch.norm(gating_reshaped * layer.weight, p=2, dim=(1,2,3)).sum()
            elif isinstance(layer, nn.Linear):
                gating_reshaped = gating.view(-1,1)
                gl = torch.norm(gating_reshaped * layer.weight, p=2, dim=(1,)).sum()
            elif isinstance(layer, nn.BatchNorm2d):
                gl = torch.norm(gating * layer.weight, p=1).sum()
            else:
                continue
            group_lasso_shared += self.gamma_shared * gl

        group_lasso_unshared = 0.0
        for i, neighbor_name in enumerate(self.unshared_layers):
            gating = self.unshared_gating_params[i]
            layer = get_submodule(self.model, neighbor_name)

            if isinstance(layer, nn.Conv2d):
                gating_reshaped = gating.view(-1,1,1,1)
                gl = torch.norm(gating_reshaped * layer.weight, p=2, dim=(1,2,3)).sum()
            elif isinstance(layer, nn.Linear):
                gating_reshaped = gating.view(-1,1)
                gl = torch.norm(gating_reshaped * layer.weight, p=2, dim=(1,)).sum()
            elif isinstance(layer, nn.BatchNorm2d):
                gl = torch.norm(gating * layer.weight, p=1).sum()
            else:
                continue
            group_lasso_unshared += self.gamma_unshared * gl

        total_loss = task_loss + alignment_loss + group_lasso_shared + group_lasso_unshared
        return total_loss

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return 100.0 * correct / total


for neighbor_id in range(1, 71):
    # Skip the local client itself if needed
    if neighbor_id == 14:
        continue

    model_path = f'./models/cifar100_client{neighbor_id}.pth'
    if not os.path.exists(model_path):
        print(f"Model for Client {neighbor_id} not found, skipping...")
        continue

    # 1) Load neighbor model
    print(f"\n=== Loading model from Client {neighbor_id} ===")
    neighbor_model = resnet18(num_classes=100).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    neighbor_model.load_state_dict(checkpoint['model_state_dict'])

    # 2) Determine shared vs. unshared layers with local_model
    shared_layers, unshared_layers = find_shared_and_unshared_layers(local_model, neighbor_model)
    print(f"  Found {len(shared_layers)} shared layers and {len(unshared_layers)} unshared layers.")

    # 3) Build the pruning model
    pruning_model = StructuredPruning(
        neighbor_model,
        shared_layers,
        unshared_layers,
        lambda_reg=0.1,
        gamma_shared=0.05,
        gamma_unshared=0.02
    ).to(device)

    # 4) Prepare optimizer/scheduler
    optimizer = optim.SGD([
        {"params": pruning_model.model.parameters()},
        {"params": pruning_model.shared_gating_params, "weight_decay": 0},
        {"params": pruning_model.unshared_gating_params, "weight_decay": 0}
    ], lr=0.01, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

    # 5) Train (prune + align) using client 14's data
    epochs = 50
    for epoch in range(epochs):
        pruning_model.train()
        epoch_loss = 0.0

        for inputs, targets in client_train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            loss = pruning_model.loss(local_model, inputs, targets)
            loss.backward()

            # Optional gradient clipping
            torch.nn.utils.clip_grad_norm_(pruning_model.parameters(), max_norm=1.0)

            optimizer.step()

            # Clamp gating parameters to [0,1] after each step
            for gating_param in pruning_model.shared_gating_params:
                gating_param.data.clamp_(0, 1)
            for gating_param in pruning_model.unshared_gating_params:
                gating_param.data.clamp_(0, 1)

            epoch_loss += loss.item()

        scheduler.step()
        avg_epoch_loss = epoch_loss / len(client_train_loader)
        print(f"[Client {neighbor_id}] Epoch {epoch+1}/{epochs} - Loss: {avg_epoch_loss:.4f}")

    # 6) Evaluate and save
    final_acc = evaluate(pruning_model, client_train_loader, device)
    print(f"  Final training accuracy on client 14 data: {final_acc:.2f}%")

    save_path = f'./pruned_models/pruned_client_{neighbor_id}.pth'
    torch.save(pruning_model.state_dict(), save_path)
    print(f"  Pruned model saved to {save_path}")

print("\nPruning and alignment complete for all neighbor models.")
